# Natural Language Processing using Doc2Vec on National Science Foundation Awards Abstracts
---
### Team:  
Jacob Noble  
Himanshu Gamit  
Shantanu Hadap


## 1. Pre-requisites: Sagemaker Jupyter Instance- ml.m5.xlarge and MphasisDeepInsightsTextSummarizer Product ARN

1. Make sure you are in North Virginia Region.
2. Go to https://aws.amazon.com/marketplace/pp/prodview-uzkcdmjuagetk and 
3. Subscribe and accept offer.
3. Wait for a confirmation email and then Continue to Configuration.
4. Select your launch method to SageMaker Console
5. Go to Configure for Amazon SageMaker console
6. Copy Prodcut arn keep it for later use. 


#### Make Sure to update product arn as highlighted below 

#### MphasisDeepInsightsTextSummarizer Product ARN

## 2. FOR CUSTOM USER ABSTRACT INPUT Update text variable section highlighted below

### Imports


In [1]:
!pip install gensim

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import gensim
from gensim.models import Doc2Vec
import pandas as pd
from sklearn.preprocessing import StandardScaler # Replace with SAS version

unable to import 'smart_open.gcs', disabling that module


In [3]:
#!rm -r data
!wget https://nsfdata.s3.amazonaws.com/nsfdataset.zip -O nsfdataset.zip
!unzip -o nsfdataset.zip -d data

--2020-04-16 18:15:29--  https://nsfdata.s3.amazonaws.com/nsfdataset.zip
Resolving nsfdata.s3.amazonaws.com (nsfdata.s3.amazonaws.com)... 52.217.39.20
Connecting to nsfdata.s3.amazonaws.com (nsfdata.s3.amazonaws.com)|52.217.39.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358656602 (342M) [application/zip]
Saving to: ‘nsfdataset.zip’

nsfdataset.zip      100%[===================>] 342.04M  31.5MB/s    in 8.7s    

2020-04-16 18:15:38 (39.5 MB/s) - ‘nsfdataset.zip’ saved [358656602/358656602]

Archive:  nsfdataset.zip
  inflating: data/nsf_proposals.csv  


In [4]:
!ls data

nsf_proposals.csv


In [5]:
#import saspy
import pandas as pd
import numpy as np


selected_cols = ['abstractText','date']
projects = pd.read_csv("data/nsf_proposals.csv", usecols = selected_cols, low_memory=False) #, nrows=30000
print ("Total Projects: ", projects.shape[0], "\nTotal Features: ", projects.shape[1])
projects.date = pd.to_datetime(projects.date.str.replace('D', 'T'))
projects = projects.sort_values('date')

Total Projects:  329321 
Total Features:  2


### Load in Data using Pandas

In [6]:
# Drop values without Abstract texts
projects = projects.dropna(how='any')

print("Max length of the abstractText:", projects.abstractText.str.len().max())
print("Min length of the abstractText:", projects.abstractText.str.len().min())
print("Avg length of the abstractText:", projects.abstractText.apply(lambda x: len(x) - x.count(" ")).mean())

words = projects.abstractText.str.split().apply(len)
print("Max words abstractText:", words.max())
projects.head()

Max length of the abstractText: 8166
Min length of the abstractText: 1
Avg length of the abstractText: 1645.861842268939
Max words abstractText: 1252


,abstractText,date
295988,Nondestructive Evaluation (NDE) is important t...,1985-08-30
307099,An Industry/University Cooperative Research Ce...,1985-08-30
284877,This research seeks to obtain a representation...,1985-09-06
177779,"This is an attempt to develop a ""solvated elec...",1986-01-16
166668,The New York State College of Ceramics at Alfr...,1986-06-03


### Setup Training Set for Doc2Vec

In [7]:
X_raw = projects.abstractText.str.lower().values
num_of_docs = len(X_raw)
print('Number of abstracts: ', num_of_docs)

Number of abstracts:  321560


#### Download Model Files

In [8]:
!wget https://nsfdata.s3.amazonaws.com/0nsf_doc2v.model -O 0nsf_doc2v.model
!wget https://nsfdata.s3.amazonaws.com/0nsf_doc2v.model.docvecs.vectors_docs.npy -O 0nsf_doc2v.model.docvecs.vectors_docs.npy
!wget https://nsfdata.s3.amazonaws.com/0nsf_doc2v.model.trainables.syn1neg.npy -O 0nsf_doc2v.model.trainables.syn1neg.npy
!wget https://nsfdata.s3.amazonaws.com/0nsf_doc2v.model.wv.vectors.npy -O 0nsf_doc2v.model.wv.vectors.npy

--2020-04-16 18:16:18--  https://nsfdata.s3.amazonaws.com/0nsf_doc2v.model
Resolving nsfdata.s3.amazonaws.com (nsfdata.s3.amazonaws.com)... 52.216.113.211
Connecting to nsfdata.s3.amazonaws.com (nsfdata.s3.amazonaws.com)|52.216.113.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628574 (19M) [application/x-www-form-urlencoded]
Saving to: ‘0nsf_doc2v.model’

0nsf_doc2v.model    100%[===================>]  18.72M  53.7MB/s    in 0.3s    

2020-04-16 18:16:18 (53.7 MB/s) - ‘0nsf_doc2v.model’ saved [19628574/19628574]

--2020-04-16 18:16:19--  https://nsfdata.s3.amazonaws.com/0nsf_doc2v.model.docvecs.vectors_docs.npy
Resolving nsfdata.s3.amazonaws.com (nsfdata.s3.amazonaws.com)... 52.216.164.27
Connecting to nsfdata.s3.amazonaws.com (nsfdata.s3.amazonaws.com)|52.216.164.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 385872128 (368M) [application/x-www-form-urlencoded]
Saving to: ‘0nsf_doc2v.model.docvecs.vectors_docs.npy’

0nsf_

### Create generator that will tokenize the training abstracts on the fly to save on memory

In [9]:
def doc_generator(input_docs_array):
    for i, doc in enumerate(input_docs_array):
        tokens = gensim.utils.simple_preprocess(doc)
        yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [10]:
X = doc_generator(X_raw)

### Create Initial Doc2Vec Model and Training

#### Skip to next section to work load in a pretrained model.  Training could take a potentially long time.

We conduct the replication to Document Embedding with Paragraph Vectors (http://arxiv.org/abs/1507.07998). In this paper, they showed only DBOW results to NSF data. So we replicate this experiments using not only DBOW but also DM.


In [11]:
%%time
#import multiprocessing
#from pprint import pprint
#cores = multiprocessing.cpu_count()

#It is training 100 epochs in 300 dimension vector space
#models = [
    # PV-DBOW 
#    Doc2Vec(dm=0, dbow_words=1, size=300, window=8, min_count=1, iter=100, workers=cores),
    # PV-DM w/average
    #Doc2Vec(dm=1, dm_mean=1, size=300, window=8, min_count=1, iter =100, workers=cores),
#]

#models[0].build_vocab(doc_generator(X_raw))
#print(str(models[0]))
#models[1].reset_from(models[0])
#print(str(models[1]))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 10 µs


In [12]:
#%%time 
#for i, model in enumerate(models):
#    model.train(doc_generator(X_raw), total_examples=len(X_raw), epochs=model.iter)
#    model.save(str(i)+'nsf_doc2v.model')

In [13]:
#print("Doc2Vec Embedding Shape:",model.docvecs.vectors_docs.shape)

In [14]:
# model.train(doc_generator(X_raw), total_examples=num_of_docs, epochs=1)
# model.save('nsf_doc2vec.model')

### Load Existing Model

In [15]:
existing_model =  '0nsf_doc2v.model' #'nsf_d2v_100.model' # Name of Existing model to load
model = gensim.models.Doc2Vec.load(str(existing_model)) # Model is assumed to be in the shared folder

### Testing Doc2Vec Model

1. Infer Embedding Vecor for a New Abstract
2. Generating Most Similar Articles based on document cluster. This is using cosine similary score.
3. Storing the most similar files to S3 bucket.


#### 1. Infer Embedding Vecor for a New Abstract

First, calculating cosine similarity of New Text using Paragraph Vector. Word Vector and Document Vector are separately stored. We have to add .docvecs after model name to extract Document Vector from Doc2Vec Model.

In [16]:
!wget https://nsfdata.s3.amazonaws.com/input.txt

--2020-04-16 18:16:44--  https://nsfdata.s3.amazonaws.com/input.txt
Resolving nsfdata.s3.amazonaws.com (nsfdata.s3.amazonaws.com)... 52.216.92.211
Connecting to nsfdata.s3.amazonaws.com (nsfdata.s3.amazonaws.com)|52.216.92.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1758 (1.7K) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.72K  --.-KB/s    in 0s      

2020-04-16 18:16:44 (113 MB/s) - ‘input.txt.3’ saved [1758/1758]



In [17]:
with open('./input.txt', 'rb') as file_stream:
    text = file_stream.read().decode('utf-8')

# FOR CUSTOM USER ABSTRACT INPUT - Update below text variable value

In [18]:
#text = """This study utilizes publicly available data from the National Science Foundation (NSF) Web Application Programming Interface (API). In this paper, various machine learning techniques are demonstrated to explore, analyze and recommend similar proposal abstracts to aid the NSF or Awardee with the Merit Review Process. These techniques extract textual context and group it with similar context. The goal of the analysis was to utilize a Doc2Vec unsupervised learning algorithms to embed NSF funding proposal abstracts text into vector space.  Once vectorized, the abstracts were grouped together using K-means clustering. These techniques together proved to be successful at grouping similar proposals together and could be used to find similar proposals to newly submitted NSF funding proposals. To perform text analysis, SAS® University Edition is used which supports SASPy, SAS® Studio and Python JupyterLab. Gensim Doc2vec is used to generate document vectors for proposal abstracts. Afterwards, document vectors were used to cluster similar abstracts using SAS® Studio KMeans Clustering Module. For visualization, the abstract embeddings were reduced to two dimensions using Principal Component Analysis (PCA) within SAS® Studio. This was then compared to a t-Distributed Stochastic Neighbor Embedding (t-SNE) dimensionality reduction technique as part of the Scikit-learn machine learning toolkit for Python.Conclusively, NSF proposal abstract text analysis can help an awardee read and improve their proposal model by identifying similar proposal abstracts from the last 24 years. It could also help NSF evaluators identify similar existing proposals that indirectly provides insights on whether a new proposal is going to be fruitful or not."""
print(text)

This study utilizes publicly available data from the National Science Foundation (NSF) Web Application Programming Interface (API). In this paper, various machine learning techniques are demonstrated to explore, analyze and recommend similar proposal abstracts to aid the NSF or Awardee with the Merit Review Process. These techniques extract textual context and group it with similar context. The goal of the analysis was to utilize a Doc2Vec unsupervised learning algorithms to embed NSF funding proposal abstracts text into vector space.  Once vectorized, the abstracts were grouped together using K-means clustering. These techniques together proved to be successful at grouping similar proposals together and could be used to find similar proposals to newly submitted NSF funding proposals. 
To perform text analysis, SAS® University Edition is used which supports SASPy, SAS® Studio and Python JupyterLab. Gensim Doc2vec is used to generate document vectors for proposal abstracts. Afterwards, 

In [19]:
test_vector = model.infer_vector(gensim.utils.simple_preprocess(text))

In [20]:
test_vector

array([ 6.82562828e-01, -2.77844161e-01, -3.82432163e-01, -1.89732462e-01,
        6.49394214e-01, -1.13833892e+00, -1.21166408e+00,  8.79729450e-01,
        9.96808290e-01, -1.09390342e+00,  7.74606317e-02,  8.12940061e-01,
        2.34025121e-01,  1.31931531e+00,  1.04793572e+00,  6.75552785e-01,
        5.95528781e-01, -3.19337279e-01,  2.45541856e-01,  8.75829518e-01,
        3.35987270e-01, -3.82613391e-01,  1.99713588e-01, -1.15263987e+00,
       -6.99651659e-01,  4.36860800e-01,  1.10605597e+00, -3.55357915e-01,
        2.16153443e-01, -1.07333824e-01,  1.35348177e+00,  3.09221238e-01,
        9.06548277e-02,  2.64715075e-01,  1.61698043e+00, -1.11064404e-01,
        1.29366469e+00,  8.58210176e-02,  8.75645638e-01, -1.21472515e-01,
       -1.05213082e+00,  4.88076061e-01,  7.85005927e-01,  1.90077424e+00,
        1.08122349e+00,  7.67614543e-01,  2.42246002e-01,  9.50537980e-01,
       -3.95221673e-02,  5.71659207e-01,  6.59440458e-01, -1.89894646e-01,
        4.92445230e-01,  

#### 2. Generating Most Similar Articles based on document cluster. This is using cosine similary score.

In [21]:
similar = model.docvecs.most_similar(
    positive=[test_vector], 
    topn=3)
print(similar)

[(281437, 0.27616167068481445), (277163, 0.266385555267334), (119952, 0.2662033438682556)]


In [22]:
X_raw[similar[1][0]]

'intelligent systems that say, recommend music or movies based on past interests, or recognize faces or handwriting based on labeled samples, often learn from examples using "supervised learning."  the system tries to find a prediction function: a combination of feature values of the song, movie, image, or pen movements, that, on known inputs, produces score values that agree with known preferences. some combinations may add with simple positive or negative weight parameters (the more guitar the better, or i really don\'t want accordion), while others can be more complex (nether too loud nor too soft).  if parameters for such a function can be found, then it can be hoped that, on a new input, the function will be a good approximation for the preference. \n\nin scientific computing, there are many optimization techniques used to find the best parameters.  the type called "gradient methods" is like a group hike that gets caught in the hills after dark; the members want to go downhill to 

3. Storing the most similar files to S3 bucket.

In [23]:
# file = open('top1.txt', 'w',encoding="utf-8")
# file.write(X_raw[similar[0][0]])
# file.close()

In [24]:
# file = open('top2.txt', 'w',encoding="utf-8")
# file.write(X_raw[similar[1][0]])
# file.close()

In [25]:
# file = open('top3.txt', 'w',encoding="utf-8")
# file.write(X_raw[similar[2][0]])
# file.close()

In [26]:
import boto3
s3_conn = boto3.client("s3")
s3 = boto3.resource('s3')
object = s3.Object('nsfdata', 'top1.txt')
object.put(Body=X_raw[similar[0][0]])

{'ResponseMetadata': {'RequestId': 'DB2E5EBEFE244C97',
  'HostId': 'saXGs5Bk+IOuaOruV/FsDrcgJZnr8lOYIkDxeKT9QGAMVd8e2dzdhj+6yjlIikigJuW2RZo71OY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'saXGs5Bk+IOuaOruV/FsDrcgJZnr8lOYIkDxeKT9QGAMVd8e2dzdhj+6yjlIikigJuW2RZo71OY=',
   'x-amz-request-id': 'DB2E5EBEFE244C97',
   'date': 'Thu, 16 Apr 2020 18:16:46 GMT',
   'etag': '"0ecd0c3db9b152d84f1a90c6a7bc7cfb"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"0ecd0c3db9b152d84f1a90c6a7bc7cfb"'}

###  MphasisDeepInsightsTextSummarizer

### Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [27]:
import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role

sess = sage.Session()
role = get_execution_role()

## Create Model

Now we use the Model Package to create a model,

# PLEASE CHANGE MphasisDeepInsightsTextSummarizer Product ARN

## Make sure you update model_package_arn to product arn which you retrieved from marketplace-text-summarizer


In [28]:
# Please use the appropriate ARN obtained after subscribing to the model to define 'model_package_arn'
##OHIO REGION
#model_package_arn = 'arn:aws:sagemaker:us-east-2:057799348421:model-package/marketplace-text-summarizer-11-d2490248e8de20f24ae3b72d0d74654c'

##North Virginia Region
model_package_arn = 'arn:aws:sagemaker:us-east-1:865070037744:model-package/marketplace-text-summarizer-11-d2490248e8de20f24ae3b72d0d74654c'

from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()
model = ModelPackage(model_package_arn=model_package_arn,
                    role = role,
                    sagemaker_session = sagemaker_session)

## Input File

Now we pull a sample input file for testing the model.

In [29]:
top1_txt="s3://nsfdata/top1.txt"

## Batch Transform Job

Now let's use the model built to run a batch inference job and verify it works.

In [30]:
import json 
import uuid
transformer = model.transformer(2, 'ml.m5.xlarge')
transformer.output_path = "s3://nsfdata/summerized-abstract"
transformer.transform(top1_txt, content_type='text/plain')
transformer.wait()
print("Batch Transform complete")

.................................
.2020-04-16T18:22:14.369:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
2020-04-16T18:22:14.369:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [16/Apr/2020 18:22:14] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [16/Apr/2020 18:22:14] "#033GET /execution-parameters HTTP/1.1#033" 404 -
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [16/Apr/2020 18:22:14] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [16/Apr/2020 18:22:14] "#033GET /execution-parameters HTTP/1.1#033" 404 -
2020-04-16T18:2

## Output from Batch Transform

Note: Ensure that the following package is installed on the local system : boto3

In [31]:
import boto3
print(transformer.output_path)
bucketFolder = transformer.output_path.rsplit('/')[3]
#print(s3bucket,s3prefix)
s3_conn = boto3.client("s3")

s3://nsfdata/summerized-abstract


In [32]:
bucket_name="nsfdata"
with open('result.txt', 'wb') as f:
    s3_conn.download_fileobj(bucket_name,bucketFolder+'/top1.txt.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [33]:
with open('./result.txt', 'rb') as file_stream:
    output_text = file_stream.read().decode('utf-8')

#### Original Text Input

In [34]:
print(text)

This study utilizes publicly available data from the National Science Foundation (NSF) Web Application Programming Interface (API). In this paper, various machine learning techniques are demonstrated to explore, analyze and recommend similar proposal abstracts to aid the NSF or Awardee with the Merit Review Process. These techniques extract textual context and group it with similar context. The goal of the analysis was to utilize a Doc2Vec unsupervised learning algorithms to embed NSF funding proposal abstracts text into vector space.  Once vectorized, the abstracts were grouped together using K-means clustering. These techniques together proved to be successful at grouping similar proposals together and could be used to find similar proposals to newly submitted NSF funding proposals. 
To perform text analysis, SAS® University Edition is used which supports SASPy, SAS® Studio and Python JupyterLab. Gensim Doc2vec is used to generate document vectors for proposal abstracts. Afterwards, 

#### Similar abstract

In [35]:
bucket_name="nsfdata"
with open('top1.txt', 'wb') as f:
    s3_conn.download_fileobj(bucket_name,'top1.txt', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [36]:
with open('./top1.txt', 'rb') as file_stream:
    top1_text = file_stream.read().decode('utf-8')

In [37]:
print(top1_text)

brown university proposes a project to create a flexible, language agnostic software library that makes text and block programming tools accessible to visually-impaired users.  screen-readers, the most common affordable tool for presenting text to those who can't read it directly, have difficulties presenting source code which is symbol-heavy and navigated best not as a list of words, but as an abstract syntax tree (ast). this project will produce a collection of javascript libraries that annotate source code with structural descriptions suitable for presentation by screen readers. 

javascript was chosen because web-based environments address common technical and logistical challenges in schools. they place fewer demands on it staff for installation and maintenance, and enable students to work outside of school without manually transferring files. web-based software is also easier to upgrade and evolve on the developer's end. being the lingua-franca for cloud-based uis, the browser's 

#### Summary of similar abstract

In [38]:
print(output_text)

Brown university proposes a project to create a flexible , language agnostic software library that makes text and block programming tools accessible to visually-impaired users. This project will produce a collection of javascript libraries that annotate source code with structural descriptions suitable for presentation by screen readers. Javascript was chosen because web-based environments address common technical and logistical challenges in schools.
Execution time : 1.50seconds

